<a href="https://colab.research.google.com/github/Tzoll87/Codes/blob/master/Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv(r'train.csv', encoding = 'latin-1')

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df['rating_num'].value_counts(dropna = False).sort_values()

2.0      17
3.0      17
0.0      40
5.0      71
9.0     144
6.0     149
3.5     239
10.0    291
8.0     310
8.5     398
6.5     424
7.5     491
7.0     548
Name: rating_num, dtype: int64

In [ ]:
df.shape

(3139, 800)

In [ ]:
df.head()

,row_id,scout_id,rating_num,winner,team,competitionId,player_position_1,player_position_2,player_height,player_weight,...,team2_defensive_derived_var_10,team2_offensive_derived_var_11,team2_offensive_derived_var_12,team2_offensive_derived_var_13,team2_offensive_derived_var_14,team2_defensive_derived_var_15,team2_offensive_derived_var_16,team2_defensive_derived_var_17,team2_offensive_derived_var_18,team2_offensive_derived_var_19
0,1,13,7.0,winner,team1,8,7.0,7.0,0.317073,0.48,...,0.193658,0.131548,0.097131,0.600944,0.338892,0.347980,0.034157,0.288962,0.310727,0.204925
1,3,16,6.5,loser,team2,8,3.0,9.0,0.463415,0.42,...,0.190500,0.173697,0.272801,0.605659,0.526761,0.468976,0.054034,0.334962,0.380073,0.295392
2,4,4,8.5,loser,team1,5,11.0,11.0,0.682927,0.44,...,0.312314,0.250756,0.212631,0.558833,0.686523,0.441749,0.219441,0.414288,0.570757,0.437500
3,5,13,8.0,loser,team2,4,17.0,17.0,0.682927,0.58,...,0.282556,0.155969,0.206503,0.533002,0.373959,0.527733,0.095201,0.423040,0.330754,0.235558
4,7,11,3.5,draw,team1,5,10.0,10.0,0.731707,0.70,...,0.450453,0.199050,0.134002,0.688773,0.681724,0.733625,0.128719,0.448360,0.444946,0.312500


In [ ]:
df.columns

Index(['row_id', 'scout_id', 'rating_num', 'winner', 'team', 'competitionId',
       'player_position_1', 'player_position_2', 'player_height',
       'player_weight',
       ...
       'team2_defensive_derived_var_10', 'team2_offensive_derived_var_11',
       'team2_offensive_derived_var_12', 'team2_offensive_derived_var_13',
       'team2_offensive_derived_var_14', 'team2_defensive_derived_var_15',
       'team2_offensive_derived_var_16', 'team2_defensive_derived_var_17',
       'team2_offensive_derived_var_18', 'team2_offensive_derived_var_19'],
      dtype='object', length=800)

In [ ]:
df['player_position_1'].value_counts(dropna = False)

3.0     507
11.0    419
2.0     292
1.0     257
12.0    192
4.0     191
17.0    179
7.0     171
13.0    148
5.0     145
8.0     143
9.0     117
16.0    116
10.0     79
6.0      79
15.0     72
19.0     10
14.0      9
18.0      7
NaN       6
Name: player_position_1, dtype: int64

In [ ]:
df.describe(exclude = 'number')

,winner,team
count,3139,3139
unique,3,2
top,winner,team1
freq,1416,1581


In [ ]:
df.describe()

,row_id,scout_id,rating_num,competitionId,player_position_1,player_position_2,player_height,player_weight,player_general_var_0,player_general_var_1,...,team2_defensive_derived_var_10,team2_offensive_derived_var_11,team2_offensive_derived_var_12,team2_offensive_derived_var_13,team2_offensive_derived_var_14,team2_defensive_derived_var_15,team2_offensive_derived_var_16,team2_defensive_derived_var_17,team2_offensive_derived_var_18,team2_offensive_derived_var_19
count,3139.000000,3139.000000,3139.000000,3139.000000,3133.000000,3133.000000,3138.000000,3138.000000,3139.000000,3139.000000,...,2952.000000,3124.000000,2951.000000,2952.000000,2952.000000,2952.000000,2952.000000,2952.000000,2952.000000,2952.000000
mean,2147.705957,10.043963,7.172348,7.238930,7.586020,7.969358,0.497925,0.455978,0.031029,0.004779,...,0.346946,0.181985,0.254402,0.578770,0.481570,0.500347,0.113668,0.425446,0.424007,0.326163
std,1237.449195,5.589242,1.833906,2.312904,5.014413,4.872416,0.163768,0.142628,0.082923,0.068973,...,0.139893,0.059080,0.140739,0.153385,0.164405,0.137598,0.063533,0.096441,0.121442,0.140226
min,1.000000,1.000000,0.000000,4.000000,1.000000,1.000000,0.024390,0.040000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.023931,0.103969,0.006881,0.092370,0.062625,0.000000
25%,1069.500000,3.000000,6.500000,5.000000,3.000000,3.000000,0.390244,0.340000,0.000000,0.000000,...,0.243458,0.159937,0.155576,0.482991,0.363758,0.409620,0.067467,0.357651,0.339094,0.231269
50%,2163.000000,12.000000,7.500000,7.000000,7.000000,8.000000,0.512195,0.440000,0.000000,0.000000,...,0.337038,0.184843,0.240096,0.590122,0.473675,0.502753,0.102874,0.421710,0.423921,0.305790
75%,3213.500000,15.000000,8.500000,9.000000,11.000000,11.000000,0.609756,0.560000,0.000000,0.000000,...,0.428551,0.216206,0.337837,0.686451,0.597440,0.596598,0.148759,0.487980,0.509561,0.404761
max,4301.000000,26.000000,10.000000,15.000000,19.000000,19.000000,0.975610,0.940000,0.800000,1.000000,...,0.962835,1.000000,0.955624,0.988499,0.989602,0.912862,1.000000,0.904275,0.903851,0.937784


In [ ]:
missings = []
for item in df.columns:
    if df[item].isnull().sum() > 14000:
        missings.append(item)

In [ ]:
todrop1 = []
useful = []
for data in missings:
    value = df[data].nunique()
    if (value < 20) & (value != 0):
        useful.append(data)
        print(data, ' has ', value, 'unique values')
    else:
        todrop1.append(data)

In [ ]:
for item in df[useful]:
    print(df[item].value_counts(dropna = False))

In [ ]:
for item in useful:
    minim = df[df[item].isnull() == False] 
    minim = minim.groupby('rating_num').count()[item].reset_index()
    print(minim)
    plt.bar(x=minim['rating_num'], height=minim[item])
    plt.show()

In [ ]:
df = df.drop(useful, axis = 1)

In [ ]:
looksgood = []
problematic = []
issue = []
for item in df.columns:
    if df[item].isnull().sum() < 1000:
        print(item, "has ", df[item].isnull().sum(), " missing values")
        looksgood.append(item)
    elif (df[item].isnull().sum() > 1000) & (df[item].isnull().sum() > 5000):
        problematic.append(item)
    else:
        issue.append(item)

row_id has  0  missing values
scout_id has  0  missing values
rating_num has  0  missing values
winner has  0  missing values
team has  0  missing values
competitionId has  0  missing values
player_position_1 has  6  missing values
player_position_2 has  6  missing values
player_height has  1  missing values
player_weight has  1  missing values
player_general_var_0 has  0  missing values
player_general_var_1 has  0  missing values
player_general_var_2 has  0  missing values
player_general_var_3 has  0  missing values
player_general_var_4 has  0  missing values
player_general_var_5 has  0  missing values
player_positional_raw_var_0 has  64  missing values
player_positional_raw_var_1 has  64  missing values
player_positional_raw_var_2 has  64  missing values
player_positional_raw_var_3 has  64  missing values
player_defensive_raw_var_4 has  64  missing values
player_defensive_raw_var_5 has  64  missing values
player_other_raw_var_6 has  47  missing values
player_offensive_raw_var_7 has  

In [ ]:
print(len(looksgood))
print(len(problematic))
print(len(issue))

667
0
133


In [ ]:
modified = df.drop(todrop1, axis = 1)

In [ ]:
modified.shape

(3139, 800)

In [ ]:
modified = modified.dropna(subset = ['player_position_1'])

In [ ]:
corr = modified.corr().abs()

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=bool))
tri_df = corr.mask(mask)

In [ ]:
to_drop = [c for c in tri_df.columns if any(tri_df[c] >  0.95)]

In [ ]:
len(to_drop)

213

In [ ]:
to_drop.remove('player_position_1')

In [ ]:
modified = modified.drop(to_drop, axis = 1)

In [ ]:
modified.isnull().sum().sort_values().tail(50)

player_offensive_ratio_var_40     2242
player_other_ratio_var_66         2327
player_offensive_ratio_var_13     2456
player_positional_ratio_var_22    2525
player_offensive_ratio_var_78     2560
player_offensive_ratio_var_72     2686
player_other_ratio_var_34         2701
player_other_ratio_var_33         2701
player_other_ratio_var_14         2722
player_defensive_ratio_var_16     2746
player_other_ratio_var_36         2830
player_offensive_ratio_var_77     2882
player_other_ratio_var_67         2958
player_offensive_ratio_var_69     2974
player_offensive_ratio_var_68     2994
player_defensive_ratio_var_39     3021
player_defensive_ratio_var_38     3021
player_positional_ratio_var_79    3046
player_other_ratio_var_23         3055
player_other_ratio_var_25         3065
player_other_ratio_var_28         3066
player_other_ratio_var_27         3110
player_other_ratio_var_24         3115
player_other_ratio_var_26         3118
team1_other_ratio_var_32          3133
team1_other_raw_var_38   

In [ ]:
for item in looksgood:
    try:
        modified[item] = modified[item].fillna(modified[item].mean())
    except:
        pass

In [ ]:
modified = modified.drop(problematic, axis = 1)

In [ ]:
def pos1(x):
    if x['player_position_1'] in [19.0,18.0,14.0,22.0,20.0,23.0]:
        return "Other"
    else:
        return x['player_position_1']

In [ ]:
modified['pos1_trans'] = modified.apply(lambda x: pos1(x), axis = 1)

In [ ]:
df['player_position_2'].value_counts()

11.0    419
9.0     321
3.0     306
2.0     292
1.0     257
12.0    192
4.0     190
17.0    179
7.0     171
13.0    147
5.0     144
8.0     143
16.0    115
6.0      80
10.0     79
15.0     72
19.0     10
14.0      9
18.0      7
Name: player_position_2, dtype: int64

In [ ]:
def pos2(x):
    if x['player_position_2'] in [10.0,19.0,18.0,14.0,22.0,20.0,23.0]:
        return "Other"
    else:
        return x['player_position_2']

In [ ]:
modified['pos2_tran'] = modified.apply(lambda x: pos2(x), axis = 1)

In [ ]:
dummies = pd.get_dummies(modified[['team', 'winner','pos2_tran', 'pos1_trans']], drop_first=True)
modified = pd.concat([modified.drop(['team', 'winner','pos2_tran', 'pos1_trans', 'player_position_2', 'player_position_1'],axis=1), dummies],axis=1)

In [ ]:
modified = modified.drop(['row_id', 'scout_id', 'competitionId'], axis = 1)

In [ ]:
modified['target'] = modified['rating_num']*10

In [ ]:
modified = modified.drop('rating_num', axis = 1)

In [ ]:
y = modified['target']
X = modified.drop('target', axis = 1)

In [ ]:
X = X.fillna(0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
la = LassoCV(max_iter=100000000)

In [ ]:
la.fit(X_train, y_train)
print(f'Optimal alpha = {la.alpha_:.3f}')

Optimal alpha = 0.033


In [ ]:
r_squared = la.score(X_test, y_test)
print(r_squared)

0.07933357215376335


In [ ]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

# Print the importances per feature
print(dict(zip(X.columns, rf.feature_importances_.round(2))))

# Print accuracy
print(f"{acc:.1%} accuracy on test set.") 

{'player_height': 0.0, 'player_weight': 0.0, 'player_general_var_1': 0.0, 'player_general_var_2': 0.0, 'player_general_var_3': 0.0, 'player_general_var_4': 0.0, 'player_general_var_5': 0.0, 'player_positional_raw_var_0': 0.0, 'player_positional_raw_var_1': 0.0, 'player_positional_raw_var_2': 0.0, 'player_positional_raw_var_3': 0.0, 'player_defensive_raw_var_4': 0.0, 'player_defensive_raw_var_5': 0.0, 'player_other_raw_var_6': 0.0, 'player_offensive_raw_var_7': 0.0, 'player_other_raw_var_8': 0.0, 'player_other_raw_var_9': 0.0, 'player_other_raw_var_10': 0.0, 'player_offensive_raw_var_11': 0.0, 'player_other_raw_var_12': 0.0, 'player_other_raw_var_13': 0.0, 'player_other_raw_var_14': 0.0, 'player_positional_raw_var_15': 0.0, 'player_positional_raw_var_16': 0.0, 'player_positional_raw_var_17': 0.0, 'player_other_raw_var_18': 0.0, 'player_other_raw_var_19': 0.0, 'player_other_raw_var_20': 0.0, 'player_positional_raw_var_21': 0.0, 'player_positional_raw_var_22': 0.0, 'player_positional_raw_

In [ ]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.0015

# Prints out the mask
print(mask)

[ True  True False False False  True False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False  True False False
 False False False  True  True False False False False False False False
 False False False False False False False False False  True False False
  True  True  True  True False False False False False False False False
 False False False False False False  True False False False  True False
 False False False False False False  True  True  True  True  True  True
  True  True False False  True  True  True False  True False  True False
 False  True  True  True False  True  True False False  True  True False
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False Fa

In [ ]:
# Apply the mask to the feature dataset X
reduced_X = X.loc[:, mask]

# prints out the selected column names
print(reduced_X.columns)

Index(['player_height', 'player_weight', 'player_general_var_4',
       'player_other_raw_var_6', 'player_offensive_raw_var_31',
       'player_other_raw_var_38', 'player_other_raw_var_44',
       'player_other_raw_var_45', 'player_other_raw_var_63',
       'player_defensive_raw_var_66',
       ...
       'team2_defensive_derived_var_10', 'team2_offensive_derived_var_11',
       'team2_offensive_derived_var_12', 'team2_offensive_derived_var_13',
       'team2_offensive_derived_var_14', 'team2_defensive_derived_var_15',
       'team2_offensive_derived_var_16', 'team2_defensive_derived_var_17',
       'team2_offensive_derived_var_18', 'team2_offensive_derived_var_19'],
      dtype='object', length=319)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reduced_X, y, test_size = 0.3)

In [ ]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

# Print accuracy
print(f"{acc:.1%} accuracy on test set.") 

40.4% accuracy on test set.
